# This module for calculating different IOUs 
For every function u need the following things
- matrix `[numpy] [torch]`
- pair     `[numpy] [torch]`
- pair loss `[numpy][torch]`

In [ ]:
#| default_exp bbox_func/bbox_iou

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export 
import torch 
import numpy as np
import fastcore.all as fc

In [ ]:
import torchvision

In [ ]:
#| export 
def _upcast(t: torch.Tensor) -> torch.Tensor:
    # Protects from numerical overflows in multiplications by upcasting to the equivalent higher type
    if t.dtype not in (torch.float32, torch.float64):
        return t.float()
    return t

In [ ]:
#| export 
def check_2d_3d(shape: int):
    if shape == 6: return 3
    if shape == 4: return 2
    raise NotImplementedError("Only 2D and 3D bboxes are defined")

In [ ]:
fc.eq(check_2d_3d(6), 3)
fc.eq(check_2d_3d(4), 2)
fc.test_fail(check_2d_3d, args=dict(shape=1))

# IOU - Numpy

In [ ]:
#| export 
def calculate_iou_numpy(pred_bbox: np.ndarray, gt_bbox: np.ndarray):
    """
    Calculate Intersection over Union (IoU) between two sets of bounding boxes using numpy broadcasting.
    :param pred_bbox: numpy array
                      An (Nx4/Nx6) array of predicted bounding boxes in the format [xmin, ymin, xmax, ymax]/[xmin, ymin, zmin, xmax, ymax, zmax]
    :param gt_bbox: numpy array
                      An (Mx4/Mx6) array of ground truth bounding boxes in the format [xmin, ymin, xmax, ymax]/[xmin, ymin, zmin, xmax, ymax, zmax]
    :return iou: numpy array
                      An (NxM) array containing IoU value between each predicted bounding box and ground truth bounding box.
    """
    dim = check_2d_3d(pred_bbox.shape[-1])
    x1 = np.maximum(pred_bbox[:, None, :dim], gt_bbox[:, :dim])
    x2 = np.minimum(pred_bbox[:, None, dim:], gt_bbox[:, dim:])
    inter = np.maximum(0, x2 - x1)
    inter_area = np.prod(inter, axis=-1)
    pred_area = np.prod(pred_bbox[:, -dim:] - pred_bbox[:, :dim], axis=1)
    gt_area = np.prod(gt_bbox[:, -dim:] - gt_bbox[:, :dim], axis=1)
    union = pred_area[:, None] + gt_area - inter_area
    iou = inter_area / union
    return np.clip(iou, 0, 1)

### test_iou_2d 

In [ ]:
pred_bbox = np.array([[0, 0, 10, 10], [10, 10, 20, 20]])
gt_bbox = np.array([[5, 5, 15, 15], [15, 15, 25, 25]])
expected_output = np.array([[0.1428, 0.0], [0.1428, 0.1428]])
fc.test_close(calculate_iou_numpy(pred_bbox, gt_bbox), expected_output, eps=1e-2)

### test_iou_3d

In [ ]:
pred_bbox = np.array([[0, 0, 0, 10, 10, 10], [10, 10, 10, 15, 15, 15]])
gt_bbox = np.array([[5, 5, 5, 15, 15, 15], [15, 15, 15, 17, 17, 17]])
expected_output = np.array([[0.0667, 0.0], [0.125, 0.0]])
fc.test_close(calculate_iou_numpy(pred_bbox, gt_bbox), expected_output, eps=1e-2)

In [ ]:
calculate_iou_numpy(pred_bbox[0][None], gt_bbox[0][None])

array([[0.06666667]])

## IOU - torch

In [ ]:
xy = torch.tensor([[0, 0, 10, 10], [10, 10,  15, 15 ]])
yx = torch.tensor([[5, 5, 15, 15], [12, 12,  17, 17 ]])

In [ ]:
xy1 = torch.max(xy[:, :2], yx[:, :2])
xy2 = torch.min(xy[:, 2:], yx[:, 2:])
xy1, xy2

(tensor([[ 5,  5],
         [12, 12]]),
 tensor([[10, 10],
         [15, 15]]))

In [ ]:
inter_iou = torch.prod(xy2-xy1, dim=-1)
inter_iou

tensor([25,  9])

In [ ]:
xyarea = torch.prod(xy[:, 2:] - xy[:, :2] , dim=-1)
yxarea = torch.prod(yx[:, 2:] - yx[:, :2] , dim=-1)

In [ ]:
yxarea

tensor([100,  25])

In [ ]:
#| export 
COMPUTE_DTYPE = torch.float32

In [ ]:
#| export 
def intersection_area_pair(b1: torch.Tensor, b2: torch.Tensor, dim: int=2):
    x1 = torch.max(b1[:, :dim], b2[:, :dim])
    x2 = torch.min(b1[:, dim:], b2[:, dim:])
    inter_hw = torch.clamp(_upcast(x2 - x1), min=0)
    inter = torch.prod(inter_hw, dim=-1)
    return inter

In [ ]:
#| export 
def bbox_area(b: torch.Tensor, dim: int=2):
    return torch.prod(_upcast(b[:, dim:] - b[:, :dim]), dim=-1)

In [ ]:
#| export     
def calculate_iou_pair(b1: torch.Tensor, b2: torch.Tensor):
    """calculate pairwaise iou score. bbox1: N, 4/6, bbox2: N, 4/6"""
    assert b1.shape == b2.shape , "b1 and b2 are of not the same shape"
    dim = check_2d_3d(b1.shape[1])
    inter = intersection_area_pair(b1, b2, dim)
    b1_area, b2_area = bbox_area(b1, dim), bbox_area(b2, dim)
    union = (b1_area + b2_area - inter)
    iou = inter/ (union+torch.finfo(COMPUTE_DTYPE).eps)
    return iou

In [ ]:
calculate_iou_pair(xy[0].unsqueeze(0), yx[0].unsqueeze(0))

tensor([0.1429])

In [ ]:
torchvision.ops.box_iou(xy, yx)

tensor([[0.1429, 0.0000],
        [0.2500, 0.2195]])

In [ ]:
#| export 
def intersection_area(b1: torch.Tensor, b2: torch.Tensor, dim: int=2):
    x1 = torch.max(b1[:, None, :dim], b2[:, :dim])
    x2 = torch.min(b1[:, None, dim:], b2[:, dim:])
    inter = torch.clamp(_upcast(x2 - x1), min=0)
    inter_area = torch.prod(inter, dim=-1)
    return inter_area

In [ ]:
#| export 
def calculate_iou_torch(b1: torch.Tensor, b2: torch.Tensor):
    """
    Calculate Intersection over Union (IoU) between two sets of bounding boxes using PyTorch broadcasting.
    :param b1: torch tensor
                      A (Nx4/Nx6) tensor of predicted bounding boxes in the format [xmin, ymin, xmax, ymax]/[xmin, ymin, zmin, xmax, ymax, zmax]
    :param b2: torch tensor
                      A (Mx4/Mx6) tensor of ground truth bounding boxes in the format [xmin, ymin, xmax, ymax]/[xmin, ymin, zmin, xmax, ymax, zmax]
    :return iou: torch tensor
                      A (NxM) tensor containing IoU value between each predicted bounding box and ground truth bounding box.
    """
    dim = check_2d_3d(b1.shape[-1])
    inter_area = intersection_area(b1, b2, dim)
    b1_area, b2_area = bbox_area(b1, dim), bbox_area(b2, dim)
    union = b1_area[:, None] + b2_area - inter_area
    iou = inter_area / (union+torch.finfo(COMPUTE_DTYPE).eps)
    return iou.clamp(min=0, max=1)

### Test-IOU-Torch

In [ ]:
pred_bbox = torch.tensor([[0, 0, 0, 10, 10, 10], [10, 10, 10, 15, 15, 15]])
gt_bbox = torch.tensor([[5, 5, 5, 15, 15, 15], [15, 15, 15, 17, 17, 17]])
expected_output = torch.tensor([[0.0667, 0.0], [0.125, 0.0]])
fc.test_close(calculate_iou_torch(pred_bbox, gt_bbox), expected_output, eps=1e-2)

In [ ]:
pred_bbox = torch.tensor([[0, 0, 10, 10], [10, 10, 15, 15]])
gt_bbox = torch.tensor([[5, 5, 15, 15], [15, 15, 17, 17]])
fc.test_close(torchvision.ops.box_iou(pred_bbox, gt_bbox), calculate_iou_torch(pred_bbox, gt_bbox), eps=1e-2)

In [ ]:
x = torch.hstack([torch.randint(20, size=(1000, 1)) for _ in range(3)])
y = torch.Tensor([[40, 40, 40] for i in range(1000)])
xy = torch.hstack([x, y])
xy.shape

torch.Size([1000, 6])

In [ ]:
nxy = xy.numpy()
nxy.shape

(1000, 6)

In [ ]:
%time _ = calculate_iou_numpy(nxy, nxy)

CPU times: user 58.3 ms, sys: 9.76 ms, total: 68.1 ms
Wall time: 67 ms


In [ ]:
%time _ = calculate_iou_torch(xy, xy)

CPU times: user 3.42 s, sys: 103 ms, total: 3.52 s
Wall time: 256 ms


In [ ]:
if torch.cuda.is_available(): xy = xy.cuda()

In [ ]:
%time _ = calculate_iou_torch(xy, xy)

CPU times: user 4.95 ms, sys: 4.34 ms, total: 9.29 ms
Wall time: 12.5 ms


> torch is faster than numpy

> cuda is faster in torch but moving tensors to cuda is taking time. 

In [ ]:
#| export 
calculate_iou = fc.TypeDispatch([calculate_iou_torch, calculate_iou_numpy])

# [DIOU](https://arxiv.org/pdf/1911.08287.pdf)
we will only implement torch version and it should work for both 2d and 3d. 


while calculating how close two bounding boxes are iou is only one aspect. we should also look into other things like 
- `overlap area`
- `central point distance` and 
- `aspect ratio`

In this DIOU case we will add `central point distance` as a negative term to diou.

This is important as IOU between two bboxes which doesn't have overlap is always zero irrespective of their distance. 

$$
R_{DIOU} = \frac{\rho^2(b, b^{gt})}{c^2}
$$
  
$$
L_{DIoU} = 1 - IOU + R_{DIOU},
$$
    

- where $\rho(.)$ is the eculidean distance between b and $b^{gt}$,
- c is the diagonal length of the samllest enclosing box covering the two boxes

In [ ]:
xy = torch.Tensor([[10, 10, 30, 30], 
                   [15, 15, 25, 25], 
                   [24, 24, 28, 28], 
                   [40, 40, 80, 80],
                   [5, 5, 35, 35]])
yx = xy.flipud()
yx

tensor([[ 5.,  5., 35., 35.],
        [40., 40., 80., 80.],
        [24., 24., 28., 28.],
        [15., 15., 25., 25.],
        [10., 10., 30., 30.]])

In [ ]:
xy, yx

(tensor([[10., 10., 30., 30.],
         [15., 15., 25., 25.],
         [24., 24., 28., 28.],
         [40., 40., 80., 80.],
         [ 5.,  5., 35., 35.]]),
 tensor([[ 5.,  5., 35., 35.],
         [40., 40., 80., 80.],
         [24., 24., 28., 28.],
         [15., 15., 25., 25.],
         [10., 10., 30., 30.]]))

In [ ]:
#| export 
def cal_diou_pair(b1: torch.Tensor, b2: torch.Tensor):
    """where b1 and b2 have same shape N x 4/6"""
    dim = check_2d_3d(b1.shape[1])
    iou = calculate_iou_pair(b1, b2)
    
    ## center Distance between the bounding boxes
    b1_ctrs = (b1[:,  dim:] + b1[:, :dim])/2
    b2_ctrs = (b2[:,  dim:] + b2[:, :dim])/2
    rho_sq = (_upcast(b1_ctrs - b2_ctrs)**2).sum(1)
    
    ## min-enclosing bbox diagnoal distance. 
    xc = torch.min(b1[:, :dim], b2[:, :dim])
    yc = torch.max(b1[:, dim:], b2[:, dim:])
    diag_sq = (_upcast(yc - xc)**2).sum(1)
    
    diou = iou - (rho_sq/(diag_sq+ torch.finfo(COMPUTE_DTYPE).eps))
    return diou

In [ ]:
%time iou = cal_diou_pair(xy, yx)
iou

CPU times: user 1.21 ms, sys: 449 µs, total: 1.66 ms
Wall time: 1.56 ms


tensor([ 0.4444, -0.3787,  1.0000, -0.3787,  0.4444])

In [ ]:
torchvision.ops.distance_box_iou(xy, yx)

tensor([[ 0.4444, -0.3265, -0.0500,  0.2500,  1.0000],
        [ 0.1111, -0.3787, -0.2043,  1.0000,  0.2500],
        [-0.0222, -0.3686,  1.0000, -0.2043, -0.0500],
        [-0.2844,  1.0000, -0.3686, -0.3787, -0.3265],
        [ 1.0000, -0.2844, -0.0222,  0.1111,  0.4444]])

In [ ]:
xy[0], yx[0]

(tensor([10., 10., 30., 30.]), tensor([ 5.,  5., 35., 35.]))

In [ ]:
calculate_iou_pair(xy[0].unsqueeze(0), yx[0].unsqueeze(0))

tensor([0.4444])

In [ ]:
torchvision.ops.box_iou(xy[0].unsqueeze(0), yx[0].unsqueeze(0))

tensor([[0.4444]])

In [ ]:
#| export 
def cal_diou(b1: torch.Tensor, b2: torch.Tensor):
    """calculating DIOU between two matrixs"""
    dim = check_2d_3d(b1.shape[1])
    iou = calculate_iou_torch(b1, b2)
    
    ## center Distance between the bounding boxes
    b1_ctrs = (b1[:,  dim:] + b1[:, :dim])/2
    b2_ctrs = (b2[:,  dim:] + b2[:, :dim])/2
    rho_sq = (_upcast(b1_ctrs[:, None, :] - b2_ctrs)**2).sum(2)
    
    ## min-enclosing bbox diagnoal distance. 
    xc = torch.min(b1[:, None,  :dim], b2[:, :dim])
    yc = torch.max(b1[:, None, dim:], b2[:, dim:])
    diag_sq = (_upcast(yc - xc)**2).sum(2)
    
    diou = iou - (rho_sq/(diag_sq+ torch.finfo(COMPUTE_DTYPE).eps))
    return diou

In [ ]:
%time iou = cal_diou(xy, yx)
iou

CPU times: user 504 µs, sys: 189 µs, total: 693 µs
Wall time: 583 µs


tensor([[ 0.4444, -0.3265, -0.0500,  0.2500,  1.0000],
        [ 0.1111, -0.3787, -0.2043,  1.0000,  0.2500],
        [-0.0222, -0.3686,  1.0000, -0.2043, -0.0500],
        [-0.2844,  1.0000, -0.3686, -0.3787, -0.3265],
        [ 1.0000, -0.2844, -0.0222,  0.1111,  0.4444]])

In [ ]:
fc.test_close(torchvision.ops.distance_box_iou(xy, yx), cal_diou(xy, yx), eps=1e-2)
fc.test_close(torchvision.ops.distance_box_iou(xy[0].unsqueeze(0), yx[0].unsqueeze(0)), \
              cal_diou(xy[0].unsqueeze(0), yx[0].unsqueeze(0)), eps=1e-2)

# GIoU
we will only implement torch version and it should work for both 2d and 3d. 


need area of convex hull enclosing the pair of boxes in question apart from iou

In [ ]:
#| export 
def cal_giou_pair(b1: torch.Tensor, b2: torch.Tensor):
    """where b1 and b2 have same shape N x 4/6"""
    dim = check_2d_3d(b1.shape[1])
    inter = intersection_area_pair(b1, b2, dim)
    b1_area, b2_area = bbox_area(b1, dim), bbox_area(b2, dim)
    union = (b1_area + b2_area - inter)
    iou = inter/ (union+torch.finfo(COMPUTE_DTYPE).eps)
    convex_hull_size = torch.max(b1[:, dim:], b2[:, dim:]) - torch.min(b1[:, :dim], b2[:, :dim])
    enc = convex_hull_size.prod(dim=1)
    giou = iou - (1-union/enc)
    return giou

In [ ]:
b1 = torch.Tensor(np.asarray([[1, 1, 4, 4, 7, 7], [2, 2, 5, 5, 8, 8], [3, 3, 6, 6, 9, 9]]))
b2 = torch.Tensor(np.asarray([[2, 2, 4, 4, 6, 6], [3, 3, 5, 5, 7, 7], [1, 1, 6, 6, 8, 8]]))

In [ ]:
dim = check_2d_3d(b1.shape[1])

In [ ]:
convex_hull_size = torch.max(b1[:, dim:], b2[:, dim:]) - torch.min(b1[:, :dim], b2[:, :dim])
convex_hull_size


tensor([[3., 6., 3.],
        [3., 6., 3.],
        [5., 8., 3.]])

In [ ]:
enc = convex_hull_size.prod(dim=1)
enc

tensor([ 54.,  54., 120.])

In [ ]:
inter = intersection_area_pair(b1, b2, dim)
inter

tensor([16., 16., 30.])

In [ ]:
b1_area, b2_area = bbox_area(b1, dim), bbox_area(b2, dim)
union = (b1_area + b2_area - inter)
union

tensor([54., 54., 94.])

In [ ]:
iou = inter/ (union+torch.finfo(COMPUTE_DTYPE).eps)
iou

tensor([0.2963, 0.2963, 0.3191])

In [ ]:
giou = iou - (1-union/enc)
giou

tensor([0.2963, 0.2963, 0.1025])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()